# Stan w programowaniu obiektowym

## Zmienne instancyjne i klasowe, zasady dostępu do danych

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@o2.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

# Sonda Mars Climate Orbiter

<img src="img/mars.jpg" width="700"/>

Przyczyną katastrofy sondy były użycie amerykańskich jednostek miar (funtów) w modelu odpowiedzialnym za obliczanie trajektorii lotu sondy. Pozostałe moduły używały jednostek SI i ciąg obliczany był w niutonach. Ta różnica doprowadziła do niewłaściwej korekty lotu i w efekcie katastrofy. Koszt misji wyniósł 326 milnionów dolarów.

# Program strukturalny

```c
struct SpaceProbe {
    double spaceProbeSpeed;
    double spaceProbeWeight;
}

double thrustCorrectionForX(struct SpaceProbe spaceProbe){
    // operowanie na polach spaceProbeSpeed oraz spaceProbeWeight
}

double thrustCorrectionForY(struct SpaceProbe spaceProbe){
    // ...
}

double thrustCorrectionForZ(struct SpaceProbe spaceProbe){
    // ...
}
```

# Primitive obsession

```java
class SpaceProbe {
    private double speedX;
    private double speedY;
    private double speedZ;
    private double weight;
    
    public double thrustCorrectionForX(double positionX, 
                      double positionY, double positionZ){
        //...
    }
}
```

Używanie typów prymitywnych tam, gdzie mamy do czynienia z wartościami domenowymi. Tutaj składowe wektora prędkości są osobnymi polami i nie mają jednostki. Masa też nie ma jednostki.

# Klasa `Speed`

```java
class Speed {
    public int value;
    public String unit;
    
    public Speed(int value, String unit){
        this.value = value;
        this.unit = unit;
    }
}
```

* Klasa `Speed` definiuje dwa **pola**: value oraz unit. Służą one do przechowywania stanu obiektów klasy Speed. 
* Oba pola są *publiczne* oznacza to, że można je odczytywać i zapisywać *na zewnątrz* klasy. Oznacza to dokładnie, że każda inna klasa ma dostęp do tych pól.
* Klasa posiada również publiczny konstruktor, który akceptuje dwa parametry. Te parametry wykorzystywane są do
  *inicjalizacji* obiektów klasy `Speed`.
* W konstruktorze występuje słowo kluczowe `this` - pozwala ono odróżnić zmienne lokalne `value` i `unit` od 
  zmiennych *instancyjne* `value` i `unit`. Poprzedzenie nazwy zmiennej tym słowem, oznacza, że odnosimy się 
  do zmiennych, których właścicielem jest obiekt.
* Jedna składowa wektora typu int została wprowadzona dla uproszczenia przykładu.

```java
Speed speed1 = new Speed(10, "km/h");
Speed speed2 = new Speed(20, "m/s");
```

* Dalej widzimy wykorzystanie konstruktora - słowo `new` służy do tworzenia nowych obiektów. 
  Wywołuje ono konstruktor z takimi samymi parametrami jak te zdefiniowane w klasie.
* Wywołanie `speed1.value = 20` zmienia **stan** obiektu `speed1`.

<img src="img/instance_variables.png" />

```java
speed1.value = 20;
speed2.unit = "km/h";
```

<img src="img/instance_variables_2.png"/>

```java
speed1.value = -10;
speed2.unit = "ala ma kota";
```

<img src="img/public_1.png"/>

```java
enum SpeedUnit {
    MS,
    KMH
}
```

```java
class Speed {
    public int value;
    public SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}
```

```java
Speed speed1 = new Speed(10, SpeedUnit.KMH);
Speed speed2 = new Speed(20, SpeedUnit.MS);
```

<img src="img/instance_variables_5.png"/>

# Modyfikator `private`

```java
class SpaceProbe {
    private Speed speed;
    
    public SpaceProbe(Speed speed){
        this.speed = speed;
    }
    
    public getSpeed(){
        return this.speed;
    }
}
```

```java
SpaceProbe probe1 = new SpaceProbe(new Speed(10, SpeedUnit.KMH));

probe1.speed = new Speed(10, SpeedUnit.MS); // niedozwolone, pole jest prywatne!
```

```java
Speed speed1 = probe1.getSpeed();

speed1.unit = SpeedUnit.MS;         // zmienia się jednostka prędkości sondy!
```

W Javie wszystkie wartoście z wyjątkiem prymitywnych są zawsze przekazywane przez referencję.

# Modyfikator `final` - `Speed` jako ValueObject

```java
class Speed {
    public final int value;
    public final SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}
```

```java
SpaceProbe probe1 = newSpaceProbe(new Speed(10, SpeedUnit.KMH));
Speed speed1 = probe1.getSpeed();

speed1.unit = SpeedUnit.MS;                 // niedozwolone!!!

```

 <img src="img/room_temperature.png" width="300"/>

* ValueObject nie może zmienić swojej wartości - jest jak liczba, znak, etc. Przykładem może być konkretna temperatura 20 stopni C.
* Inny obiekt korzystający z ValueObject może zminić wartość *parametru wyrażanego za pomocą* ValueObject. Np. temperatura pokoju moze zmienić się z 20 na 22. Ale same valueObjects się nie zmienią. 

# Modyfikator `private`
```java
class Speed {
    private int value;
    private SpeedUnit unit;

    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}
```

```java
class SpaceProbe
    private Speed speed;
    
    public void accelerate(Speed delta){
        speed.value += delta.value;                              // niedozwolone!
    }
}
```

* Obiekty "zewnętrzne" nie mogą **odczytywać** ani **zapisywać** wartości pól prywatnych. Ale co to znaczy?

```java
class Speed {
    public Speed add(Speed that){
        if(this.unit == that.unit){
            return new Speed(this.value + that.value, this.unit);
        } else {
            return null;
        }
    }
}
```

```java
class SpaceProbe
    private Speed speed;

    public void accelerate(Speed delta){
        Speed newSpeed = this.speed.add(delta);
        if(newSpeed){
            this.speed = newSpeed;
        }
    }
}
```

<img src="img/private.png"/>

# Modyfikator `protected`

```java
class Speed {
    protected SpeedUnit unit;
    
    public Speed(SpeedUnit unit){
        this.unit = unit;
    }
}
```

```java
class Speed1D extends Speed {
    protected int value;

    public Speed1D(int value, SpeedUnit unit){
        super(unit);
        this.value = value
    }
    
    public Speed1D add(Speed1D delta){
        if(this.unit == delta.unit){
            return new Speed1D(this.value + delta.value, this.unit);
        } else {
            return null;
        }
    }
}
```

```java
Speed1D speed1 = new Speed1D(10, SpeedUnit.KMH);
Speed1D speed2 = new Speed1D(20, SpeedUnit.KMH);

Speed1D speed3 = speed1.add(speed2);
```

```java
class Speed3D extends Speed {
    protected int valueX;
    protected int valueY;
    protected int valueZ;

    public Speed3D(int x, int y, int z, SpeedUnit unit){
        super(value);
        this.valueX = x;
        this.valueY = y;
        this.valueZ = z;
    }
    
    public Speed3D add(Speed3D delta){
        if(this.unit == delta.unit){
            return new Speed3D(this.valueX + delta.valueX, 
                this.valueY + delta.valueY,
                this.valueZ + delta.valueZ, 
                this.unit);
        } else {
            return null;
        }
    }
}
```

<img src="img/protected.png"/>

# Dostęp pakietowy

```java
package agh.cs.lecture;

class Speed {
    int value;
    SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
}

```

```java
package agh.cs.lecture;

class SpaceProbe {
    private Speed speed;
    
    public void accelerate(Speed delta){
        if(this.speed.unit == delta.unit){           // dozwolone
            //..
        }
    }
}
```

```java
package com.mycompany;

class SpaceShip {
    private Speed speed;

    public void accelerate(Speed delta){
        if(this.speed.unit == delta.unit){           // niedozwolone!
            //..
        }
    }
}
```

<img src="img/package_access.png"/>

# Metody dostępowe - "gettery"

```java
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public int getValue(){
        return this.value;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}
```

# Dostęp dziedzinowy

```java
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public int getValueInMS(){
        if(this.unit == SpeedUnit.MS){
            return value;
        } else {
            return convert(this.unit, SpeedUnit.MS, this.value);
        }
    }
    
    public int getValueInKMH(){
        //...
    }
    
    private int convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.KMS && to == SpeedUnit.MS)
            return Math.round(value / 3.6);
        } else if(...){
            //...
            
        }
    }
}
```

# Metody dostępowe - "settery"

```java
class Speed {
    private int value;
    private SpeedUnit unit;
    
    public setValue(int value){
        this.value = value;
    }
    
    public setUnit(SpeedUnit unit){
        this.value = convert(this.unit, unit, this.value);
        this.unit = unit;
    }
}
```

To już nie jest "value object" !!

# Utrzymywanie jednolitej reprezentacji

```java
class Speed {
    private int valueInMs;
    private SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.valueInMs = convert(unit, SpeedUnit.MS, value);
        this.unit = unit;
    }
    
    public getValue(){
        return convert(SpeedUnit.MS, this.unit, this.valueInMs);
    }
    
    public setValue(int value){
        this.valueInMs = convert(unit, SpeedUnit.MS, value);
    }
    
    public setUnit(SpeedUnit unit){
        this.unit = unit;
    }
}
```

# ValueObject - konwersja

```java
class Speed {
    private final int value;
    private final SpeedUnit unit;
    
    public Speed(int value, SpeedUnit unit){
        this.value = value;
        this.unit = unit;
    }
    
    public Speed convertToMs(){
        if(this.unit == SpeedUnit.MS){
            return this;
        } else {
            return new Speed(convert(this.unit, SpeedUnit.MS, this.value), 
                             SpeedUnit.MS);
        }
    }
}
```

# Shadowing

```java
enum SpeedUnit { MS, KMH }
```

```java
class Speed {
    private SpeedUnit unit;
    
    public Speed(SpeedUnit unit){
        this.unit = unit;
    }
    
    public SpeedUnit getSuperUnit(){
        return this.unit;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}
```

```java
class Speed1D extends Speed{
    private SpeedUnit unit;
    private int value;
    
    public Speed1D(int value, SpeedUnit unit){
        super(unit);
        this.value = value;
        this.unit = SpeedUnit.KMH;
    }
    
    public SpeedUnit getUnit(){
        return this.unit;
    }
}
```

```java
Speed1D speed1 = new Speed1D(10, SpeedUnit.MS);
speed1.getUnit();
speed1.getSuperUnit();
```

<img src="img/shadowing.png"/>

# Zmienne statyczne

```java
class Speed {
    private int value;
    private SpeedUnit unit;
    
    private int convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * 3.6;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / 3.6;
        } else {
            //...
        }
    }
}
```

```java
class Speed {
    private final double ms2kmhRatio = 3.6;
    
    private int value;
    private SpeedUnit unit;
    
    private int convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * ms2kmhRatio;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / ms2kmhRatio;
        } else {
            //...
        }
    }
}
```

<img src="img/static1.png"/>

# Modyfikator `static`

```java
class Speed {
    private static final double ms2kmhRatio = 3.6;
    
    private int value;
    private SpeedUnit unit;
    
    private int convert(SpeedUnit from, SpeedUnit to, int value){
        if(from == SpeedUnit.MS && to == SpeedUnit.KMH){
            return value * ms2kmhRatio;
        } else if(from == SpeedUnit.KMH && to == SpeedUnit.MS){
            return value / ms2kmhRatio;
        } else {
            //...
        }
    }
}
```

<img src="img/static2.png"/>

# Antywzorzec

```java
class SpaceShip {
    public static List<SpaceShip> ships = new LinkedList<>();
    
    public SpaceShip(){
        ships.add(this);
    }
}
```

```java
class Space() {
    private List<SpaceShip> ships = new LinkedList<>();
    
    public void add(SpaceShip ship){
        ships.add(ship);
    }
}

class SpaceShip {
    public SpaceShip(Space space){
        space.add(this);
    }
}
```

# Model prywatności w Ruby

```ruby
class Speed
  def initialize(value, unit)
    @value = value
    @unit = unit
  end
  
  def to_s
    "#{@value} #{@unit}"
  end
  
  def +(other)
    Speed.new(@value + other.@value, @unit + other.@unit) # niepoprawna składnia!
  end
end
```

<img src="img/ruby_variable.png" />

```ruby
class Speed
  attr_reader :unit, :value
    
  def initialize(value, unit)
    @value = value
    @unit = unit
  end
    
  def +(other)
    Speed.new(@value + other.value, @unit + other.unit)
  end
end
```

<img src="img/ruby_variable_1.png" />

```ruby
class SpaceShip
  def initalize(speed)
    @speed = speed
  end
    
  def accelerate(speed)
    @speed += speed
  end
end
```

```ruby
class Speed
  attr_reader :unit
  
  def initialize(unit)
    @unit = unit
  end
end
```

```ruby
class Speed1D
  attr_reader :value
  
  def initialize(unit, value)
    super(unit)
    @value = value
    @unit = :KMH
  end
end
```

```ruby
speed1 = Speed1D.new(10, :MS)
speed1.unit #=> :KMH
```

<img src="img/ruby_class.png" />

![Pytania? ](http://cliparts.co/cliparts/qcB/jqg/qcBjqgxc5.jpg)

TODO
* dostęp do pól w C#
* zawirowania Javy

```java
class A {
    public int a;
    
    public A(int a){
        this.a = a;
    }
}

class B extends A{
    public int a;
    public B(int a){
        super(a);
        this.a = 5;
    }
}

A a = new B(4);
a.a;
B a = new B(4);
a.a;
```